In [2]:
import pandas as pd
import re

#
df_happy = pd.read_csv('../../../../2019happy.csv')
df_happy.columns = df_happy.columns.str.strip()
df_happy = df_happy.rename(columns={
    'Country or region': 'Country',
    'Score':             'Happiness',
    'Social support':    'Social_support'
})
df_happy['Year'] = 2019

#
df = pd.read_csv('../../../../WDI_new.csv')
df.columns = df.columns.str.strip()
df = df.rename(columns={'Country Name':'Country','Series Code':'Series_Code'})

#
year_cols = [c for c in df.columns if re.match(r'\d{4} \[YR\d{4}\]', c)]
year_map = {c: c.split(' ')[0] for c in year_cols}
df = df.rename(columns=year_map)
df_long = df.melt(
    id_vars=['Country','Series_Code'],
    value_vars=list(year_map.values()),
    var_name='Year', value_name='Value'
)
df_long['Year'] = df_long['Year'].astype(int)
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

#
wanted = {
    "NY.GDP.PCAP.KN":    "GDP_per_capita",
    "NY.GNP.PCAP.KN":    "GNI_per_capita",
    "SI.POV.GINI":       "Gini_index",
    "SP.DYN.LE00.IN":    "Life_expectancy",
    "SL.UEM.TOTL.NE.ZS": "Unemployment_rate",
    "SE.XPD.TOTL.GB.ZS": "Edu_expenditure_pct" 
}
w19 = df_long[(df_long['Year']==2019) & 
             (df_long['Series_Code'].isin(wanted))]
wide = (
    w19.pivot_table(
        index=['Country','Year'],
        columns='Series_Code',
        values='Value',
        aggfunc='first'
    )
    .rename(columns=wanted)
    .reset_index()
)

# 
df_final = pd.merge(df_happy[['Country','Year','Happiness']],
                    wide, on=['Country','Year'], how='inner')

df_final.index = df_final.index + 1
display(df_final.head())

ModuleNotFoundError: No module named 'pandas'